<h1 align='center' style=color:cyan;text-weight:800>
1. Extracción Transformación y Carga (ETL)
</h1>

## _1.1 Extracción_

In [1]:
# Carga de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

_Carga el de los archivos a utilizar en el análisis_

In [2]:
df_homicidios = pd.read_excel('./datasets/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])
df_vic_hom    = pd.read_excel('./datasets/homicidios.xlsx',sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])
df_lesiones   = pd.read_excel('./datasets/lesiones.xlsx' , sheet_name = 'HECHOS',na_values=['SD', 'sd'])
df_vic_les    = pd.read_excel('./datasets/lesiones.xlsx' , sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])

## _1.2 Transformación_

### _1.2.1 Homicidios_Hechos_

In [3]:
display(df_homicidios.sample(3))

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
567,2020-0047,1,2020-10-27,2020,10,27,14:00:00,14.0,AUTOPISTA CAMPORA ALTURA LACARRA AV.,AUTOPISTA,...,NaN,LACARRA AV.,AUTOPISTA PRESIDENTE HÃ‰CTOR J. CÃMPORA y LAC...,9,Point (99689.80114953 97506.41939342),-58.46668375,-34.65174801,PEATON-CARGAS,PEATON,CARGAS
73,2016-0098,1,2016-07-27,2016,7,27,16:08:00,16.0,AV SAN JUAN Y ALBERTI,AVENIDA,...,NaN,ALBERTI,SAN JUAN AV. y ALBERTI,3,Point (105765.97394356 100631.47718024),-58.40042405,-34.62356174,MOTO-CARGAS,MOTO,CARGAS
51,2016-0074,1,2016-06-12,2016,6,12,06:40:00,6.0,AV GRAL PAZ Y RODOLFO SCAPINO,GRAL PAZ,...,NaN,"SCAPINO, RODOLFO","PAZ, GRAL. AV. y SCAPINO, RODOLFO",8,Point (96731.44252651 94668.88351657),-58.49896559,-34.67732118,AUTO-AUTO,AUTO,AUTO


_Removemos las columnas que no vamos a utilizar_

In [4]:
df_homicidios.drop(columns=['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 
                            'Altura', 'Cruce', 'Dirección Normalizada', 'PARTICIPANTES','VICTIMA'], inplace=True)

In [5]:
# INFOREME DE HOMICIDIOS
print(df_homicidios.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             695 non-null    float64       
 4   TIPO_DE_CALLE  696 non-null    object        
 5   COMUNA         696 non-null    int64         
 6   XY (CABA)      696 non-null    object        
 7   pos x          696 non-null    object        
 8   pos y          696 non-null    object        
 9   ACUSADO        673 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 54.5+ KB
None


In [6]:
# Cantidad de valores nulos
df_homicidios.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
XY (CABA)         0
pos x             0
pos y             0
ACUSADO          23
dtype: int64

_Transformaciones de datos_

In [7]:
# remplazar la comuna 0 a nulos
df_homicidios.loc[df_homicidios['COMUNA'] == '0' ,'COMUNA'] = np.nan


# remplazar los "." por nulos en latitud y longitud
df_homicidios.loc[df_homicidios['pos x'] == '.' ,'pos x'] = np.nan
df_homicidios.loc[df_homicidios['pos y'] == '.' ,'pos y'] = np.nan
df_homicidios.loc[df_homicidios['XY (CABA)'] == 'Point (. .)' ,'XY (CABA)'] = np.nan


In [8]:
df_homicidios.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
XY (CABA)        14
pos x            12
pos y            12
ACUSADO          23
dtype: int64

In [9]:
# cambio a formato categórico
df_homicidios['HH'] = df_homicidios['HH'].astype('category')
df_homicidios['TIPO_DE_CALLE'] = df_homicidios['TIPO_DE_CALLE'].astype('category')
df_homicidios['COMUNA'] = df_homicidios['COMUNA'].astype('category')
df_homicidios['ACUSADO'] = df_homicidios['ACUSADO'].astype('category')

# cambio a float los valores de latitud y longitud
df_homicidios['pos x'] = df_homicidios['pos x'].astype('float')
df_homicidios['pos y'] = df_homicidios['pos y'].astype('float')

df_homicidios.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             695 non-null    category      
 4   TIPO_DE_CALLE  696 non-null    category      
 5   COMUNA         696 non-null    category      
 6   XY (CABA)      682 non-null    object        
 7   pos x          684 non-null    float64       
 8   pos y          684 non-null    float64       
 9   ACUSADO        673 non-null    category      
dtypes: category(4), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 37.4+ KB


Rename de columnas

In [10]:
nuevos_nombres = {'ID': 'ID_HECHO','pos x':'LONGITUD','pos y':'LATITUD',
                  'HH':'HORA','ACUSADO':'VEHICULO_ACUSADO',
                  'Rango_etario ':'RANGO_ETARIO'}
                 
# Remplazar nombres
df_homicidios = df_homicidios.rename(columns=nuevos_nombres)

In [11]:
# # borrando los registros nulos que son 15 que representa el 2.1% de los datos
# df_homicidios.isnull().sum()
# df_homicidios.dropna(inplace=True)

_Guardamos el DF listo para su análisis_

In [12]:
df_homicidios.sample(3)

,ID_HECHO,N_VICTIMAS,FECHA,HORA,TIPO_DE_CALLE,COMUNA,XY (CABA),LONGITUD,LATITUD,VEHICULO_ACUSADO
23,2016-0034,1,2016-03-12,12.0,AVENIDA,4.0,Point (105230.98315750 97613.97258373),-58.406239,-34.650765,PASAJEROS
538,2020-0018,1,2020-03-09,14.0,AVENIDA,13.0,Point (101309.75947342 109680.73260642),-58.449031,-34.542003,OBJETO FIJO
11,2016-0016,1,2016-02-14,22.0,AVENIDA,4.0,Point (108661.59115206 98481.75445555),-58.368827,-34.642920,AUTO


In [13]:
# Guardamos el DF listo para su análisis
df_homicidios.to_csv('./datasets/df_homicidios.csv',index=False)

### _1.2.2 Homicidios_Victimas_

In [14]:
display(df_vic_hom.sample(3))

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
24,2016-0035,2016-03-13,2016,3,13,CONDUCTOR,MOTO,MASCULINO,34.0,2016-03-13 00:00:00
595,2020-0056,2020-11-21,2020,11,21,CONDUCTOR,MOTO,FEMENINO,34.0,2020-11-21 00:00:00
150,2017-0009,2017-01-16,2017,1,16,CONDUCTOR,MOTO,MASCULINO,36.0,2017-01-16 00:00:00


In [15]:
print(df_vic_hom.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB
None


_Removemos las columnas que no vamos a utilizar_

In [16]:
df_vic_hom.drop(columns=['AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO'], inplace=True)

df_vic_hom['GRAVEDAD'] = 'FATAL'

_Transformaciones de datos_

In [17]:
# función que regresa 1 si la edad es menor a 18, 2 si la edad esta entre 18 y 29
# 3 si la edad esta entre 30 y 39, 4 si la edad esta entre 40 a 49 , 5 si la edad es de 50 a 59
# y 6 si la edad es igual o mayor a 60 
def rango_etario(edad):
    if edad < 18:
        return "1_ 0 a 17 años"
    elif 18 <= edad <= 29:
        return "2_ 18 a 29 años"
    elif 30 <= edad <= 39:
        return "3_ 30 a 39 años"
    elif 40 <= edad <= 49:
        return "4_ 45 a 49 años"
    elif 50 <= edad <= 59:
        return "5_ 50 a 59 años"
    elif edad >= 60:
        return "6_ 60 años o mas"
    else:
        return np.nan


In [18]:
# Rango Etario
df_vic_hom.loc[df_vic_hom['EDAD'] == 'SD', 'EDAD'] = np.nan
df_vic_hom.loc[df_vic_hom['EDAD'] == 'sd', 'EDAD'] = np.nan
df_vic_hom['EDAD'] = df_vic_hom['EDAD'].astype('float')
df_vic_hom['RANGO_ETARIO'] = df_vic_hom['EDAD'].apply(lambda x: rango_etario(x))
df_vic_hom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID_hecho      717 non-null    object        
 1   FECHA         717 non-null    datetime64[ns]
 2   ROL           706 non-null    object        
 3   VICTIMA       708 non-null    object        
 4   SEXO          711 non-null    object        
 5   EDAD          664 non-null    float64       
 6   GRAVEDAD      717 non-null    object        
 7   RANGO_ETARIO  664 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 44.9+ KB


In [19]:
# cambio a formato categórico
df_vic_hom['ID_hecho'] = df_vic_hom['ID_hecho'].astype('category')
df_vic_hom['ROL'] = df_vic_hom['ROL'].astype('category')
df_vic_hom['VICTIMA'] = df_vic_hom['VICTIMA'].astype('category')
df_vic_hom['SEXO'] = df_vic_hom['SEXO'].astype('category')
df_vic_hom['RANGO_ETARIO'] = df_vic_hom['RANGO_ETARIO'].astype('category')


df_vic_hom.drop(columns='EDAD',inplace=True)

df_vic_hom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID_hecho      717 non-null    category      
 1   FECHA         717 non-null    datetime64[ns]
 2   ROL           706 non-null    category      
 3   VICTIMA       708 non-null    category      
 4   SEXO          711 non-null    category      
 5   GRAVEDAD      717 non-null    object        
 6   RANGO_ETARIO  664 non-null    category      
dtypes: category(5), datetime64[ns](1), object(1)
memory usage: 38.0+ KB


In [20]:
# CAMBIAR EL NOMBRE A LAS COLUMNAS QUE NO ESTAN EN MAYUSCULAS   
nuevos_nombres = {'ID_hecho':'ID_HECHO','ROL':'ROL_VICTIMA','VICTIMA':'VEHICULO_VICTIMA','SEXO':'SEXO_VICTIMA'}

# Remplazar nombres
df_vic_hom = df_vic_hom.rename(columns=nuevos_nombres)
df_vic_hom.columns

Index(['ID_HECHO', 'FECHA', 'ROL_VICTIMA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA',
       'GRAVEDAD', 'RANGO_ETARIO'],
      dtype='object')

In [21]:
#cambia el ROL_VICTIMA  =  PASAJERO_ACOMPAÑANTE por PASAJERO
df_vic_hom['ROL_VICTIMA'] = df_vic_hom['ROL_VICTIMA'].replace('PASAJERO_ACOMPAÑANTE','PASAJERO')

In [22]:
# Guardamos el DF listo para su análisis
df_vic_hom.to_csv('./datasets/df_vic_hom.csv',index=False)

### _1.2.3 Lesiones_Hechos_

In [23]:
display(df_lesiones.sample(3))

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
3902,LC-2019-0299771,1,2019,5,14,2019-05-14 00:00:00,08:10:00,8.0,NaN,9,...,-34.634635,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
1829,LC-2019-0153063,1,2019,3,12,2019-03-12 00:00:00,20:06:00,20.0,NaN,12,...,-34.565084,NaN,NaN,SD-SD,x,NaN,NaN,NaN,NaN,NaN
1076,LC-2019-0095110,1,2019,2,13,2019-02-13 00:00:00,19:38:00,19.0,NaN,3,...,-34.605645,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print(df_lesiones.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23776 non-null  object 
 7   franja_hora            23776 non-null  float64
 8   direccion_normalizada  12917 non-null  object 
 9   comuna                 22770 non-null  object 
 10  tipo_calle             12740 non-null  object 
 11  otra_direccion         5490 non-null   object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

_Removemos las columnas que no vamos a utilizar_

In [25]:
df_lesiones.drop(columns=['aaaa', 'mm', 'dd','hora','direccion_normalizada',
                          'calle','altura','cruce','participantes','moto',
                          'auto','transporte_publico','camion','otra_direccion',
                          'ciclista','victima','gravedad'], inplace=True)


In [26]:
# Pasar las columnas a mayusculas
df_lesiones.columns = df_lesiones.columns.str.upper()

# CAMBIAR NOMBRE DE LA COLUMNA FRANJA HORARIA
df_lesiones = df_lesiones.rename(columns={'ID':'ID_HECHO'})
df_lesiones = df_lesiones.rename(columns={'FRANJA_HORA':'HORA'})
df_lesiones = df_lesiones.rename(columns={'TIPO_CALLE':'TIPO_DE_CALLE'})
df_lesiones = df_lesiones.rename(columns={'GEOCODIFICACION_CABA':'XY (CABA)'})
df_lesiones = df_lesiones.rename(columns={'ACUSADO':'VEHICULO_ACUSADO'})
df_lesiones = df_lesiones.rename(columns={'LATUTID':'LATITUD'})

df_lesiones.sample(3)	

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_DE_CALLE,XY (CABA),LONGITUD,LATUTID,VEHICULO_ACUSADO
8044,LC-2019-0609401,1,2019-09-28 00:00:00,14.0,1,NaN,Point (107390.14725633601483423 100044.5727541...,-58.382708,-34.628842,NaN
9663,LC-2019-0810729,1,2019-12-07 00:00:00,10.0,12,NaN,Point (95609.50898635186604224 105109.19203127...,-58.511153,-34.583204,NaN
6623,LC-2019-0499111,1,2019-08-11 00:00:00,17.0,7,NaN,Point (101965.38105788156099152 99268.41861485...,-58.441865,-34.635863,NaN


_Transformaciones_

In [32]:
df_lesiones['FECHA'].replace("'","", inplace=True)
df_lesiones['FECHA'] = df_lesiones['FECHA'].astype('datetime64[ns]')

# Remplazar "No especificada" por nulos
df_lesiones.loc[df_lesiones['COMUNA'] == 'No Especificada', 'COMUNA'] = np.nan

In [33]:
df_lesiones.columns

Index(['ID_HECHO', 'N_VICTIMAS', 'FECHA', 'HORA', 'COMUNA', 'TIPO_DE_CALLE',
       'XY (CABA)', 'LONGITUD', 'LATUTID', 'VEHICULO_ACUSADO'],
      dtype='object')

In [34]:
# Guardamos el DF listo para su análisis
df_lesiones.to_csv('./datasets/df_lesiones.csv',index=False)

### _1.2.4 Lesiones_Victimas_

In [110]:
display(df_vic_les.sample(3))


,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
10751,LC-2019-0700182,2019,11,6,2019-11-06,NaN,Mujer,24.0,NaN
13479,LC-2020-0236471,2020,5,12,2020-05-12,MOTO,Mujer,45.0,NaN
8100,LC-2019-0513293,2019,8,17,2019-08-17,NaN,Mujer,35.0,NaN


In [111]:
# Pasar las columnas a mayusculas
df_vic_les.columns = df_vic_les.columns.str.upper()

# Cambiar nombres a las columnas para coincidir con el df_homicidios
df_vic_les = df_vic_les.rename(columns={'ID HECHO':'ID_HECHO','FECHA ':'FECHA','SEXO':'SEXO_VICTIMA'})

df_vic_les.columns

Index(['ID_HECHO', 'AAA', 'MM', 'DD', 'FECHA', 'VEHICULO_VICTIMA',
       'SEXO_VICTIMA', 'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

_Removemos las columnas que no vamos a utilizar_

In [112]:
df_vic_les.drop(columns=['AAA', 'MM', 'DD'], inplace=True)

# Columnas
df_vic_les.columns


Index(['ID_HECHO', 'FECHA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA', 'EDAD_VICTIMA',
       'GRAVEDAD'],
      dtype='object')

In [113]:
# Creamos la columna ROL_VICTIMA con nulls
df_vic_les['ROL_VICTIMA'] = np.nan

# remplazar los nulos de la columna GRAVEDAD por "LEVEMENTE HERIDO"
df_vic_les.loc[df_vic_les['GRAVEDAD'].isnull(), 'GRAVEDAD'] = 'LEVEMENTE HERIDO'

df_vic_les.head(3)

,ID_HECHO,FECHA,VEHICULO_VICTIMA,SEXO_VICTIMA,EDAD_VICTIMA,GRAVEDAD,ROL_VICTIMA
0,LC-2019-0000053,2019-01-01,NaN,Varon,57.0,LEVEMENTE HERIDO,NaN
1,LC-2019-0000063,2019-01-01,NaN,NaN,NaN,LEVEMENTE HERIDO,NaN
2,LC-2019-0000079,2019-01-01,NaN,Varon,NaN,LEVEMENTE HERIDO,NaN


_Transformaciones de datos_

In [114]:
df_vic_les.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID_HECHO          27605 non-null  object        
 1   FECHA             27605 non-null  datetime64[ns]
 2   VEHICULO_VICTIMA  11333 non-null  object        
 3   SEXO_VICTIMA      25377 non-null  object        
 4   EDAD_VICTIMA      23134 non-null  float64       
 5   GRAVEDAD          27605 non-null  object        
 6   ROL_VICTIMA       0 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 1.5+ MB


In [115]:
# Rango Etario
df_vic_les['EDAD_VICTIMA'] = df_vic_les['EDAD_VICTIMA'].astype('float')
df_vic_les['RANGO_ETARIO'] = df_vic_les['EDAD_VICTIMA'].apply(lambda x: rango_etario(x))

In [116]:
# Unificar en FEMENINO y MASCULINO
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'MUJER', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Mujer', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'mujer ', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'mujer', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Mujer ', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Varon', 'SEXO_VICTIMA'] = 'MASCULINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'varon', 'SEXO_VICTIMA'] = 'MASCULINO'


In [117]:
# inprime valores unicos de la columna SEXO_VICTIMA
df_vic_les['SEXO_VICTIMA'].unique()


array(['MASCULINO', nan, 'FEMENINO'], dtype=object)

In [118]:
# Guardamos el DF listo para su análisis
df_vic_les.to_csv('./datasets/df_vic_les.csv',index=False)